In [25]:
import numpy as np
import revreaddy as rdy
import logging
import datetime
import h5py as h5
import matplotlib.cm as cm

import time
import random
import string

import matplotlib.pyplot as plt
reload(logging)
logging.basicConfig(
     format='%(asctime)s %(levelname)s: %(message)s',
     datefmt='%Y/%m/%d %I:%M:%S',
     level=logging.DEBUG
    )

In [26]:
def stamp():
    
    
    timestamp = time.strftime("%Y_%m_%d-%H_%M_%S")
    randomstamp = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(6))
    stamp = timestamp + "_" + randomstamp
    return stamp

In [31]:
def simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction):

    sim = rdy.Sim("FractionalDiffusion")
 
    sim.kt = 2.437 # kJ/mol -> ~300 K
    sim.alpha = 0.5
    sim.timestep = 0.01# ns
    sim.boxsize = boxsize# nm
    sim.delete_all_particle_types() # also deletes all interactions etc.
    sim.new_type("S", 0.01, Diffusion_S)
    sim.new_type("P", 0.01, Diffusion_P)
    sim.new_type("E", 0.01, Diffusion_E)
    sim.delete_all_reactions()
    #sim.new_enzymatic("E+S<->E+S", 0, 1, 2, intrinsikreaction, 0., reactiondistance)
    sim.delete_all_particles()
    np.random.seed()

    for _ in range(200):
         pos = np.random.random(3) * 9.999 - 4.999
         sim.add_particle(pos, 0)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 1)
         #pos = np.random.random(3) * 9.9 - 4.95
         #sim.add_particle(pos, 2)
    #for _ in range(1):   
     #   pos = np.random.random(3) * 9.9 - 4.95
      #  sim.add_particle(pos, 2)

    sim.show_world()
    sim.delete_all_observables()
    #numbers_a_name = stamp()+"numbers_a_test1.h5"
    #sim.new_particle_numbers(1, numbers_a_name, 0)
    #numbers_b_name = stamp()+"numbers_b_test1.h5"
    mean_e_name = stamp()+"mean_e_test1.h5"
    #sim.new_particle_numbers(1, numbers_b_name, 1)
    #print "NUMBERS A:", numbers_a_name
    #print "NUMBERS B:", numbers_b_name
    sim.new_mean_squared_displacement(1,mean_e_name, 0)
    sim.run(5000)
    sim.write_observables_to_file()
    sim.delete_all_observables()
    #fa = h5.File(numbers_a_name, 'r')
    #fb = h5.File(numbers_b_name, 'r')
    fc = h5.File(mean_e_name, 'r')
    return  fc #fa , fb,

In [32]:
def msd_analyt(alpha,dt,t,diffconst):
    t=np.linspace(0,t,t)
    return 2*3*((t*dt)**alpha)*diffconst

In [33]:
colors = iter(cm.rainbow(np.linspace(0, 1, 4)))
for i in [3.0]:
    boxsize=10.
    Diffusion_E=i
    Diffusion_S=i
    Diffusion_P=10
    reactiondistance=3
    intrinsikreaction=10000000000000000000000
    colornow=next(colors)
    falist=[]
    fclist=[]
    averagelist=[]
    for ii in range(1):
        fc=simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) #,fc
        #plt.plot(fa['times'], fa['particleNumbers'],color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i))
        #falist.append(fa['particleNumbers'])
        print fc.keys
        fclist.append(fc['meanSquaredDisplacements'])
        #t,c,c11=analyt(np.array(fa['times']),boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction)
        #averagelist.append(c)
        
    #averagelist=np.array(averagelist) 
    #analytnumer=averagelist.mean(axis=0)
    #falist=np.array(falist)  
    fclist=np.array(fclist)  
    #t,c,c11=analyt(np.array(fa['times']),boxsize,Diffusion_E,Diffusion_S,reactiondistance,intrinsikreaction)
    #paertnum=falist.mean(axis=0) 
    msd=fclist.mean(axis=0) 
    
    #plt.plot(fb['times'], paertnum,color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i))
    plt.plot(fc['times'], msd ,color=colornow,label="$D_S+D_E$=$%.2f$" %(2*i))
    plt.plot(np.array(range(5000))*0.01,msd_analyt(0.5,0.01,5000,3.0),label="analyt")
    

    #plt.plot(t,c,"--",color=colornow)
    #plt.plot(fc['times'],)
    
plt.ylabel('MSD)', fontsize=10)
plt.xlabel('t unitless', fontsize=10)
plt.legend()

2016/07/14 05:14:06 INFO: Run with timestep 0.01 and 5000 timesteps
2016/07/14 05:14:20 INFO: Finished after 13.834453 seconds.


Number of particles:  200
TypeId | Number of particles
====== | ===================
0      | 200                
1      | 0                  
2      | 0                  
 
UniqueId | TypeId | Position            
======== | ====== | ====================
0        | 0      | [ 3.02222264  1.1150
         |        | 2489  3.58779653]   
1        | 0      | [ 2.24253872 -3.1988
         |        | 582   2.42058088]   
2        | 0      | [-4.49292654  3.1207
         |        | 1061  2.88250953]   
3        | 0      | [ 4.81194274 -3.6001
         |        | 1671  1.64672209]   
4        | 0      | [-2.34060702  1.9753
         |        | 4458 -3.03450268]   
5        | 0      | [-4.7184182  -2.7503
         |        | 871   0.73578177]   
6        | 0      | [ 0.39859568  3.5027
         |        | 4387  1.91473772]   
7        | 0      | [ 4.57302074 -1.7602
         |        | 6733 -4.41090783]   
8        | 0      | [ 0.7959803  -0.0625
         |        | 8453  2.44470849]   
9      

In [34]:
plt.show()

In [23]:
fc=simulation(boxsize,Diffusion_E,Diffusion_S,Diffusion_P,reactiondistance,intrinsikreaction) #,fc


2016/07/14 03:21:55 INFO: Run with timestep 1.0 and 5000 timesteps
2016/07/14 03:22:10 INFO: Finished after 14.563886 seconds.


Number of particles:  200
TypeId | Number of particles
====== | ===================
0      | 200                
1      | 0                  
2      | 0                  
 
UniqueId | TypeId | Position            
======== | ====== | ====================
0        | 0      | [-0.84346734  1.7914
         |        | 6536 -3.39493096]   
1        | 0      | [ 1.95800592 -3.8955
         |        | 3577 -1.16850414]   
2        | 0      | [-2.59712948  1.0628
         |        | 3895 -4.76741603]   
3        | 0      | [ 1.73066053 -0.9698
         |        | 638  -0.16915583]   
4        | 0      | [-4.72139441  0.3337
         |        | 1976 -0.22771323]   
5        | 0      | [ 4.53365655  4.5149
         |        | 0531 -3.94682595]   
6        | 0      | [-0.11859932 -1.0600
         |        | 7298 -2.97459871]   
7        | 0      | [ 2.88097198  3.2250
         |        | 0852 -3.14701855]   
8        | 0      | [ 0.8097303  -4.2152
         |        | 028   0.74443945]   
9      

In [24]:
fc.keys()

[u'meanSquaredDisplacements',
 u'numberOfParticles',
 u'standardDeviations',
 u'standardErrors',
 u'times']